In [ ]:
import tensorflow as tf
import os
os.environ['TF_XLA_FLAGS'] = '--tf_xla_enable_xla_devices'
os.environ['TF_CPP_MIN_LOG_LEVEL']='2'
print(tf.__version__)
a = tf.constant(1.)
b = tf.constant(2.)
print(a+b)
print('GPU:', tf.test.is_gpu_available())

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array, array_to_img
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.layers import Activation, Dropout, Flatten, Dense
from tensorflow.keras import backend as K
from sklearn.metrics import accuracy_score
import numpy as np
import matplotlib.pyplot as plt
import glob, os, random

In [ ]:
# 统一定义图像像素的宽度和高度
img_width, img_height = 224, 224

# 定义训练集、验证集的图形路径（文件夹路径即可）
train_data_dir = '../input/blood-cells/dataset2-master/dataset2-master/images/TRAIN'
test_data_dir = '../input/blood-cells/dataset2-master/dataset2-master/images/TEST'

# 模型训练的参数设置
epochs = 50  # 迭代次数
batch_size = 32  # 每个批量观测数

# 图像输入维度设置
if K.image_data_format() == 'channels_first':
    input_shape = (3, img_width, img_height)
else:
    input_shape = (img_width, img_height, 3)

In [ ]:
train_datagen = ImageDataGenerator(validation_split=0.15,
                                   rescale=1. / 255,  # 重缩放因子
                                   shear_range=0.2,  # 剪切强度（以弧度逆时针方向剪切角度）
                                   zoom_range=0.2,  # 随机缩放范围
                                   vertical_flip=True,  #随机上下翻转
                                   horizontal_flip=True  # 随机水平翻转
                                  )
test_datagen = ImageDataGenerator()
train_generator = train_datagen.flow_from_directory(train_data_dir,
                                                    subset="training",
                                                    target_size=(img_width, img_height), 
                                                    batch_size=batch_size,
                                                    class_mode='categorical',  # 指定分类模式
                                                    classes=['EOSINOPHIL','LYMPHOCYTE','MONOCYTE','NEUTROPHIL']
                                                   )
val_generator = train_datagen.flow_from_directory(train_data_dir,
                                                  subset="validation",
                                                  target_size=(img_width, img_height),
                                                  batch_size=batch_size,
                                                  class_mode='categorical',  # 指定分类模式
                                                  classes=['EOSINOPHIL','LYMPHOCYTE','MONOCYTE','NEUTROPHIL']
                                                  )
test_generator = test_datagen.flow_from_directory(test_data_dir,
                                                  target_size=(img_width, img_height),
                                                  class_mode='categorical',  # 指定分类模式
                                                  classes=['EOSINOPHIL','LYMPHOCYTE','MONOCYTE','NEUTROPHIL']
                                                  ) 

In [ ]:
model = Sequential()

# 添加第一个卷积层/最大池化层
model.add(Conv2D(filters=64,
          kernel_size=(3, 3),
          input_shape=input_shape, 
          activation='relu')) 
model.add(MaxPooling2D(pool_size=(2, 2))) 

# 添加第二个卷积层/最大池化层
model.add(Conv2D(filters=128, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

# 添加第三个卷积层/最大池化层
model.add(Conv2D(filters=128, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

# 由于卷积层是 2D 空间，训练时需要将数据展平为 1D 空间
model.add(Flatten())  # 添加展平层
model.add(Dense(units=128, activation='relu'))  # 添加全连接层128个神经元
model.add(Dropout(0.5))  # 添加丢弃层，防止过拟合

# 输出层：最后一层，神经元控制输出的维度，并指定分类激活函数
model.add(Dense(units=4, activation='softmax'))  # 指定分类激活函数

model.summary()

In [ ]:
model.compile(loss='categorical_crossentropy',  # 指定损失函数类型
              optimizer='rmsprop',  # 优化器
              metrics=['accuracy'])  # 评价指标

In [ ]:
history = model.fit(train_generator,
                    epochs=epochs,
                    validation_data=val_generator,
                    callbacks=[
                        tf.keras.callbacks.EarlyStopping(
                            monitor='val_loss',
                            patience=10,
                            restore_best_weights=True
                            )
                        ]
                    )

In [ ]:
model.save('model.h5')

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
training_loss = history.history['loss']
test_loss = history.history['val_loss']
# 创建迭代数量
epoch_count = range(1, len(training_loss) + 1)
# 可视化损失历史
plt.plot(epoch_count, training_loss, 'r--')
plt.plot(epoch_count, test_loss, 'b-')
plt.legend(['Training Loss', 'Test Loss'])
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.show()

In [ ]:
train_acc = history.history['accuracy']
test_acc = history.history['val_accuracy']
epoch_counts = range(1, len(train_acc)+1)
plt.plot(epoch_counts, train_acc, 'r--', marker='^')
plt.plot(epoch_counts, test_acc, linestyle='-', marker='o', color='y')
plt.title('accuracy condition')
plt.legend(['train_acc', 'test_acc'])
plt.xlabel('epochs')
plt.ylabel('acc')

In [ ]:
val_generator.reset()
pred = model.predict_generator(generator=val_generator, 
                               steps=10,  # 代表多少个 batch_size 观测数
                               verbose=1
                               )
print(pred.shape)
pred[:10]

In [ ]:
labels = (val_generator.class_indices)
print(labels)
labels = dict((v,k) for k,v in labels.items())  
print(labels)

test_x, test_y = val_generator.__getitem__(1)

preds = model.predict(test_x)

plt.figure(figsize=(16, 16))
for i in range(16):
    plt.subplot(4, 4, i+1)
    plt.title('pred:%s / truth:%s' % (labels[np.argmax(preds[i])], labels[np.argmax(test_y[i])]))
    plt.imshow(test_x[i])